# Requests 和 正则表达式 爬取猫眼电影榜单

## 测试代码

In [1]:
import requests
import re

In [3]:
headers = {
    'Referer':'https://maoyan.com/board',
    'Origin': 'https://maoyan.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362',
    'Accept-Encoding': 'zip, deflate, br'
}
response = requests.get('https://maoyan.com/board/4?offset=0', headers=headers)
print(response.status_code)
with open('demo_file/maoyan.txt','w', encoding='utf-8') as f:
    f.write(response.text)

200


In [10]:
test = '''
<dd>
                        <i class="board-index board-index-1">11231</i>
    <a href="/films/1203" title="霸王别姬" class="image-link" data-act="boarditem-click" data-val="{movieId:1203}">
      <img src="//s3plus.meituan.net/v1/mss_e2821d7f0cfe4ac1bf9202ecf9590e67/cdn-prod/file:5788b470/image/loading_2.e3d934bf.png" alt="" class="poster-default" />
      <img data-src="https://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c" alt="霸王别姬" class="board-img" />
    </a>
    <div class="board-item-main">
      <div class="board-item-content">
              <div class="movie-item-info">
        <p class="name"><a href="/films/1203" title="霸王别姬" data-act="boarditem-click" data-val="{movieId:1203}">霸王别姬</a></p>
        <p class="star">
                主演：张国荣,张丰毅,巩俐
        </p>
<p class="releasetime">上映时间：1993-01-01</p>    </div>
    <div class="movie-item-number score-num">
<p class="score"><i class="integer">9.</i><i class="fraction">5</i></p>        
    </div>

      </div>
    </div>

                </dd>
'''

In [17]:
movie_pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?<a.*?data-src="(.*?)".*?name"><a href="(.*?)".*?>(.*?)</a></p>.*?star">\s+(.*?)\s+</p>.*?releasetime">(.*?)</p>.*?integer">(\d)\.</i>.*?fraction">(\d)</i></p>.*?</dd>', re.S)
# movie_pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*data-src="(.*?)".*?name">')
result = re.findall(movie_pattern, test)
print(result)

[('11231', 'https://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', '/films/1203', '霸王别姬', '主演：张国荣,张丰毅,巩俐', '上映时间：1993-01-01', '9', '5')]


## 正式代码

In [2]:
import requests
from requests.exceptions import RequestException
import re
import os

In [18]:
ROOT = os.path.join(os.path.realpath('.'),'demo_file','maoyan.com')
if not os.path.isdir(ROOT):
    os.makedirs(ROOT)

movie_pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?<a.*?data-src="(.*?)".*?name"><a href="(.*?)".*?>(.*?)</a></p>.*?<p class="star">\s+(.*?)\s+</p>.*?releasetime">(.*?)</p>.*?integer">(\d)\.</i>.*?fraction">(\d)</i></p>.*?</dd>', re.S)

def get_one_page(url,headers,save_to_file=True):
    try:
        # Current page
        page = url.split('=')[-1]
        # Originating the request
        response = requests.get(url,headers=headers)
        # If everything is ok, and get the response from the server
        if response.status_code == 200:
            # store the page to file
            if save_to_file:
                path = os.path.join(ROOT,"maoyan_top100_page%s.txt"%page)
                print('Current file:',path)
                with open(path, 'w', encoding='utf-8') as f:
                    f.write(response.text)
            # Return the page html
            return response.text
        return None
    except RequestException as e: # Capture Request Exception
        # If exception has happened, return nothing
        print(e)
        return None

def parse_page(html):
    result = re.findall(movie_pattern, html)
    return result

In [19]:
headers = {
    'Origin': 'https://maoyan.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362',
    'Accept-Encoding': 'zip, deflate, br'
}

if __name__ == '__main__':
    url = "https://maoyan.com/board/4?offset={}"
    referer_url = 'https://maoyan.com/board'
    with open(os.path.join(ROOT, 'result.txt'), 'a', encoding='utf-8') as f:
        for i in range(10):
            current_url = url.format(i*10)
            headers['Referer'] = referer_url
            html = get_one_page(current_url, headers)
            result = parse_page(html)
            referer_url = current_url
            print('Currrent page:', current_url)
            print('page %d, size:%d\n'%((i+1), len(result)))
            f.write('---'*15)
            f.write('\npage:%d\n'%(i+1))
            if len(result) > 0:
                for line in result:
                    f.write('No.%s, 海报链接：%s, 详细链接：https://maoyan.com%s, 片名：%s, （演员）%s, （上映时间）%s, 评分：%s.%s\n'%line)
            f.write('\n')
    print('all done!')

Current file: E:\MachineLearning\Python\Python-based-Crawler-Learning\demo_file\maoyan.com\maoyan_top100_page0.txt
Currrent page: %s https://maoyan.com/board/4?offset=0
page 1, size:10

Current file: E:\MachineLearning\Python\Python-based-Crawler-Learning\demo_file\maoyan.com\maoyan_top100_page10.txt
Currrent page: %s https://maoyan.com/board/4?offset=10
page 2, size:10

Current file: E:\MachineLearning\Python\Python-based-Crawler-Learning\demo_file\maoyan.com\maoyan_top100_page20.txt
Currrent page: %s https://maoyan.com/board/4?offset=20
page 3, size:10

Current file: E:\MachineLearning\Python\Python-based-Crawler-Learning\demo_file\maoyan.com\maoyan_top100_page30.txt
Currrent page: %s https://maoyan.com/board/4?offset=30
page 4, size:10

Current file: E:\MachineLearning\Python\Python-based-Crawler-Learning\demo_file\maoyan.com\maoyan_top100_page40.txt
Currrent page: %s https://maoyan.com/board/4?offset=40
page 5, size:10

Current file: E:\MachineLearning\Python\Python-based-Crawler-L